In [ ]:
import sys
sys.path.append('../../')
import helper

from pandas import read_excel, DataFrame, merge
from matplotlib import pyplot as plt
import seaborn as sb
import numpy as np
from scipy import stats
# import warnings 
# Warning.filterwarnings('ignore')

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import ModelCheckpoint

from sklearn.preprocessing import StandardScaler

In [ ]:
boston = read_excel("https://data.hossam.kr/E04/boston.xlsx")
bostonDF = DataFrame(boston, columns=boston.columns)
bostonDF.head(10)

In [ ]:
# 에러나도 몰라 일단 작성함
# 파이썬 어쩌구 책

# 각 칼럼이 회귀 결과에 미치는 영향이 어느 정도인지 시각화 하기 

bostonDF['PRICE'] = boston.target 
print("boston 데이터 세트 크기 : ",bostonDF.shape)
bostonDF.info()
# -> 데이터 세트 피처의 Null은 없고 모두 float형

# 산점도
fig, axs = plt.subplots(figsize=(16,8),ncols = 4,nrows=2) # 2개의 행과 4개의 열을 가진 그래프
lm_features = ['RW','ZN','INDUS','NOX','AGE','PTRATIO','LSTAT','RAD']
for i, feature in enumerate(lm_features) :
    row = int(1/4)
    col = i%4
    # seaborn 의 regplot을 이용해 산점도와 선형회귀직선을 함께 표현 
    sb.regplot(x=feature, y='PRICE',data=bostonDF,ax=axs[row][col])

'''
다른 칼럼보다 RM과 LSTAT의 PRICE 영향도가 두드러진다. RM(방 개수)는 양 방향의 선형성이 가장 크다.
즉 방의 크기가 클수록 가격이 증가하는 모습을 보여준다. 
LSTAT(하위계층의 비율)은 음 방향의 선형성이 크다. 적을수록 price가 증가.'''



In [ ]:
# LinearRegression 클래스를 이용한 회귀모델 

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

y_target = bostonDF["PRICE"]
x_data = bostonDF.drop(['PRICE'], axis=1, inplace=False)
x_train,x_test,y_train, y_test = train_test_split(x_data,y_target,test_size=0.3,random_state=777)

# 선형회귀 OLS로 학습 /예측 /평가 수행
lr = LinearRegression()
lr.fit(x_train,y_train)
y_preds = lr.predict(x_test)
mse = mean_squared_error(y_test,y_preds)
rmse = np.sqrt(mse)

print("MSEL {0:.3f}, RMSE : {1:.3f}".format(mse,rmse))

print('절편 값:' , lr.intercept_)
print('회귀계수 값:' , np.round(lr.coef_,1))
# coef_속성은 회귀계수값만 가지고 있으므로 이를 피처별 회귀 계수 값으로 다시 매핑하고, 높은 값 순으로 출력 하기 위해서 판다스 Series의 sort_values() 함수를 이용한다. 

# 회귀 계수를 큰 값 순으로 정렬하기 위해 Series로 생성, 인덱스 칼럼명에 유의 
coeff = pd.Series(data=np.round(lr.coef_,1), index=x_data.columns)
coeff.sort_values(acending=False)

#RM이 양의 값으로 회귀 계수가 가장 크며 NOX 피처의 회귀 계수 -값이 너무 큼. 최적화를 하면서 피저 codfficients의 변화도 살펴봐양ㅎ암.



